In [1]:
from LxmlSoup import LxmlSoup
import requests
import pandas as pd
import re

In [2]:
def extract_data(url):
    """
    Загружает HTML-страницу Автодиспетчера (калькулятор расстояний) и извлекает:
      - name   : название точки/маршрута,
      - length : протяжённость/длина,
      - region : регион (расположен в <td colspan="2">).
    
    Особенности:
      • Структура — табличная (таблица <table> / <tr> / <td>), а не карточки div.
      • Для 'region' используется селектор с атрибутом 'colspan' — пример адресного поиска по атрибутам.
      • Безопасная итерация: берём минимальную длину найденных списков, чтобы избежать IndexError.
    Возвращает три параллельных списка одинаковой длины.
    """
    html = requests.get(url).text
    soup = LxmlSoup(html)
    
    # Ищем ячейки таблицы по классам/атрибутам:
    # - названия точек
    names   = soup.find_all("td", class_="point_name")
    # - длины/километраж
    lengths = soup.find_all("td", class_="length")
    # - регион: отличается тем, что ячейка объединённая (colspan=2)
    regions = soup.find_all("td", colspan="2", class_="region")
    
    l_name, l_length, l_region = [], [], []
    
    # Берём минимальный размер из трёх списков — самый безопасный способ
    # синхронно пробежать по разной разметке.
    for i in range(min(len(names), len(lengths), len(regions))):
        # Защита от возможных разрывов: если элемента нет — ставим заглушку.
        name   = names[i].text()   if i < len(names)   else "No name"
        length = lengths[i].text() if i < len(lengths) else "No length"
        region = regions[i].text() if i < len(regions) else "No region"
        
        l_name.append(name)
        l_length.append(length)
        l_region.append(region)
    
    return l_name, l_length, l_region

In [3]:
# Пример: маршрут Алжир → Москва
url ='https://www.avtodispetcher.ru/distance/?from=%D0%90%D0%BB%D0%B6%D0%B8%D1%80+%28%D0%90%D0%BB%D0%B6%D0%B8%D1%80%2C+%D0%90%D0%BB%D0%B6%D0%B8%D1%80%29&to=%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0+%28%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0%2C+%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F%29&v=&vt=car&rm=110&rp=90&rs=60&ru=40&fc=8&fp=55&ov=&atn=&aup=&atr=&afd=&ab=&acb='

In [4]:
# Извлечение данных
names, lengths, regions = extract_data(url)

# Упаковка в таблицу
df_routes = pd.DataFrame(list(zip(names, lengths, regions)),
                         columns=['name', 'length', 'region'])

# Сохранение в CSV
filename = 'Routes.csv'
df_routes.to_csv(filename, index=False)

# Быстрый просмотр результата в консоли

,name,length,region
0,Алжир,,
1,Casbah,0 км,
2,Algiers Centre,0 км,
3,Agha,0 км,
4,Алжир,1 км,
...,...,...,...
907,Одинцово,4502 км,
908,Ромашково,4512 км,Московская область - Москва
909,Москва,4514 км,Москва - Московская область
910,Новоивановское,4515 км,
